<a href="https://colab.research.google.com/github/cagBRT/Machine-Learning/blob/master/HP1%20Cars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Use seaborn for pairplot
!pip install seaborn


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)


#Regression: predict fuel efficiency




### Get the data
First download the dataset.

In [0]:
dataset_path = keras.utils.get_file("auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
dataset_path

In [0]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin']
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)

dataset = raw_dataset.copy()
print("done")

In [0]:
dataset.head()

# **Data Correlation**
Can the correlation of the data help improve the model?

In [0]:
corr = dataset.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
plt.show()

### Prepare the data

###Clean the data

In [0]:
dataset.isna().sum()

For this tutorial:
1. You can drop the missing rows<br>
2. Use the information at the end of this lab for the missing data values. 

In [0]:
dataset = dataset.dropna()
dataset.isna().sum()

In [0]:
dataset.describe()

You could also look at the number of times each value appears in a column. 

In [0]:
dataset['Cylinders'].value_counts()

###Convert categorical data 

In [0]:
origin = dataset.pop('Origin')
print("done")

In [0]:
dataset.tail()

In [0]:
dataset['USA'] = (origin == 1)*1.0
dataset['Europe'] = (origin == 2)*1.0
dataset['Japan'] = (origin == 3)*1.0
dataset.tail()

###Split the data into train and test sets
Now split the dataset into a training set and a test set.

We will use the test set in the final evaluation of our model.

# **Tune the Test-Training set split**

In [0]:
#0.5 - 0.95
train_dataset = dataset.sample(frac=0.5,random_state=0)
test_dataset = dataset.drop(train_dataset.index)
print("done")

### Inspect the data


Also look at the overall statistics.<br> The stats for the training set and the test set should be similiar.

In [0]:
train_stats = train_dataset.describe()
train_stats.pop("MPG")
train_stats = train_stats.transpose()
train_stats

In [0]:
test_stats = test_dataset.describe()
test_stats.pop("MPG")
test_stats = test_stats.transpose()
test_stats

### Split features from labels

Separate the target value, or "label", from the features. This label is the value that you will train the model to predict.

In [0]:
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')
print("done")

### Normalize the data

Look again at the `train_stats` block above and note how different the ranges of each feature are.

In [0]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)
print("done")

## The model

# **Define the model**
1. Change the number of neurons in the model

2. Try different activation functions:
>sigmoid<br>
tanh<br>
relu<br>
leaky_relu<br>

In [0]:
from keras import metrics

inputs = len(train_dataset.keys())
print("number of inputs to the model = " + str(inputs))

def build_model():
  model = keras.Sequential([
    #Change the number of neurons
    #Change the activation function                      
    layers.Dense(12, activation=tf.nn.sigmoid,input_shape=([len(train_dataset.keys())]),),
    layers.Dense(12, activation=tf.nn.sigmoid),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mean_squared_error', 
                optimizer=optimizer,
                metrics=['mean_absolute_error', 'mean_squared_error'])
  return model
  print("done")

### Build the model

Let's build our model. Here, we'll use a `Sequential` model with two densely connected hidden layers, and an output layer that returns a single, continuous value. The model building steps are wrapped in a function, `build_model`, since we'll create a second model, later on.

In [0]:
model = build_model()
print("done")

In [0]:
model.summary()

### Train the model

Train the model and record the training and validation accuracy in the `history` object.

ML Vocabulary: https://towardsdatascience.com/epoch-vs-iterations-vs-batch-size-4dfb9c7ce9c9

In [0]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [MPG]')
  plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
           label = 'Val Error')
  plt.ylim([0,5])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$MPG^2$]')
  plt.plot(hist['epoch'], hist['mean_squared_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_squared_error'],
           label = 'Val Error')
  plt.ylim([0,20])
  plt.legend()
  plt.show()

# **Train the model**<br>
The number of epochs is set by the early stopping, so there is no need to tune it. <br>

<br>
But you will want to tune the batch_size

In [0]:
EPOCHS = 1000
model = build_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,batch_size=2,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop])

plot_history(history)

In [0]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=1)

print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))


### Test the model

Finally, predict MPG values using data in the testing set:

In [0]:
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])
plt.show()

Add the missing data:

1971 Ford Pinto<br>
dataset.at[32,"Horsepower"] = 100<br>
print(dataset.loc[32])<br>

1974 Ford Maverick<br>
dataset.at[126,"Horsepower"] = 75<br>
print(dataset.loc[126])<br>

1980 Renault Le Car De Luxe<br>
dataset.at[330,"Horsepower"] = 50<br>
print(dataset.loc[330])<br>

1980 Ford Mustang Cobra<br>
dataset.at[336,"Horsepower"] = 88<br>
print(dataset.loc[336])<br>

1981 Renault 18i<br>
dataset.at[354,"Horsepower"] = 81<br>
print(dataset.loc[354])br>
1982 AMC Concord DL<br>
dataset.at[374,"Horsepower"] = 82<br>
print(dataset.loc[374])